In [1]:
# 1) event broker (observer design pattern)
# 2) command-query separation (cqs)
# 3) observer

from abc import ABC
from enum import Enum

In [2]:
class Event(list):
    def __call__(self, *args, **kwargs):
        for item in self:
            item(*args, **kwargs)

In [3]:
class WhatToQuery(Enum):
    ATTACK = 1
    DEFENSE = 2

class Query:
    def __init__(self, creature_name, what_to_query, default_value):
        self.value = default_value  # bidirectional
        self.what_to_query = what_to_query
        self.creature_name = creature_name

In [4]:
# event broker
class Game:
    def __init__(self):
        self.queries = Event()

    def perform_query(self, sender, query):
        self.queries(sender, query)

In [5]:
class CreatureModifier(ABC):
    def __init__(self, game, creature):
        self.creature = creature
        self.game = game
        self.game.queries.append(self.handle)

    def handle(self, sender, query):
        pass

    def __enter__(self):
        print('enter')
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        print('exit')
        self.game.queries.remove(self.handle)


class DoubleAttackModifier(CreatureModifier):
    def handle(self, sender, query):
        if (sender.name == self.creature.name and
                query.what_to_query == WhatToQuery.ATTACK):
            query.value *= 2

class IncreaseDefenseModifier(CreatureModifier):
    def handle(self, sender, query):
        if (sender.name == self.creature.name and
                query.what_to_query == WhatToQuery.DEFENSE):
            query.value += 3

In [6]:
class Creature:
    def __init__(self, game, name, attack, defense):
        self.initial_defense = defense
        self.initial_attack = attack
        self.name = name
        self.game = game

    @property
    def attack(self):
        q = Query(self.name, WhatToQuery.ATTACK, self.initial_attack)
        self.game.perform_query(self, q)
        return q.value

    @property
    def defense(self):
        q = Query(self.name, WhatToQuery.DEFENSE, self.initial_attack)
        self.game.perform_query(self, q)
        return q.value
        
    def __str__(self):
        return f'{self.name} ({self.attack}/{self.defense})'

In [7]:
game = Game()
goblin = Creature(game, 'Strong Goblin', 2, 2)
print(goblin)
DoubleAttackModifier(game, goblin)
print(goblin)

Strong Goblin (2/2)
Strong Goblin (4/2)


In [8]:
game = Game()
goblin = Creature(game, 'Strong Goblin', 2, 2)
print(goblin)
# call __enter__
with DoubleAttackModifier(game, goblin):
    print(goblin)

print(goblin)

Strong Goblin (2/2)
enter
Strong Goblin (4/2)
exit
Strong Goblin (2/2)
